<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Prokka-Comparison/blob/main/Overlaps_between_BATH%2C_Prokka%2C_Prodigal_annotations_for_binned_and_unbinned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gffpandas
!sudo apt-get install bedops
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.8/178.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gffpandas: filename=gffpandas-1.2.0-py2.py3-none-any.whl size=6247 sha256=0ad0772048f07afa9f16343922968dbac26320e2f09497ed18315ec563257213
  Stored in directory: /root/.cache/pip/wheels/57/87/f1/1d0c74fbc5151562ba7953dc110a7d8c63c6c3229d025bc8cd
Successfully built gffpandas
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjansson4 tcsh
The following NEW packages will be installed:
  bedops libjansson4 tcsh
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,922 kB of archives.
After this operation, 10.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjansson4 amd64 2.13.1-1.1build3 [32.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tcsh amd64 6.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import subprocess
import matplotlib.pyplot as plt
import gffpandas.gffpandas as gffpd
from Bio import SeqIO
from glob import glob
from pathlib import Path

In [3]:
#Input file paths
prokka_annotations = '/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/prokka_binned_unbinned_Oct7.xlsx'
bath_dedup_annotations = '/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/dedup_combinned_binned_unbinned_sep30.xlsx'
list_of_bathsearch_outputs = glob("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/BATH_Deduplication/BATHOUTPUT/*")
#Output files
prokka_annotations_save = '/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/Prokka Output/Prokka_proteins.xlsx'
bath_dedup_annotations_save = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/Deduplication Output/BATH_proteins.xlsx"
bath_prokka_alignment = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/Aligned Outputs/Aligned_Outputs.xlsx"
bathsearch_output_file = "bathsearch_combined_file.csv"

In [6]:
prokka_proteins = pd.read_excel(prokka_annotations)
print(prokka_proteins.shape)
prokka_proteins = prokka_proteins[prokka_proteins['type'] == "CDS"]
print(prokka_proteins.shape)
unnamed_cols = [col for col in prokka_proteins.columns if col.startswith('Unnamed')]
prokka_proteins.drop(columns=unnamed_cols, inplace=True)

(75516, 21)
(75516, 21)


In [10]:
# Convert the 'start' and 'end' columns to integers if they are not already
prokka_proteins['start'] = prokka_proteins['start'].astype(str)
prokka_proteins['end'] = prokka_proteins['end'].astype(str)

# Filter the DataFrame based on the conditions
filtered_prokka_proteins = prokka_proteins[
    (prokka_proteins['seq_id'] == "NODE_16543_length_5404_cov_2.090484_bin.67") &
    (prokka_proteins['start'] == "22") &
    (prokka_proteins['end'] == "174")
]

# Display the result
print(filtered_prokka_proteins)

                                           seq_id           source type start  \
0      NODE_16543_length_5404_cov_2.090484_bin.67  Prodigal:002006  CDS    22   
28071  NODE_16543_length_5404_cov_2.090484_bin.67  Prodigal:002006  CDS    22   

       end score strand  phase  \
0      174     .      +      0   
28071  174     .      +      0   

                                              attributes              ID Name  \
0      ID=GLCIONJA_00001;inference=ab initio predicti...  GLCIONJA_00001  NaN   
28071  ID=GLCIONJA_28252;inference=ab initio predicti...  GLCIONJA_28252  NaN   

      db_xref eC_number gene                             inference  \
0         NaN       NaN  NaN  ab initio prediction:Prodigal:002006   
28071     NaN       NaN  NaN  ab initio prediction:Prodigal:002006   

            locus_tag note               product coverage interval bin status  
0      GLCIONJA_00001  NaN  hypothetical protein           1.9-2.1     binned  
28071  GLCIONJA_28252  NaN  hypothetic

In [11]:
prokka_proteins_no_duplicates = prokka_proteins.drop_duplicates(subset=['seq_id', 'start', 'end'], keep='first')


In [12]:
#Load and prep bath files
bath_protein = pd.read_excel(bath_dedup_annotations)
unnamed_cols = [col for col in bath_protein.columns if col.startswith('Unnamed')]
bath_protein.drop(columns=unnamed_cols, inplace=True)
bath_protein['start'] = bath_protein['ali from']
bath_protein['end'] = bath_protein['ali to']
bath_protein['start'].fillna(bath_protein['ali from flip'],inplace=True)
bath_protein['end'].fillna(bath_protein['ali to flip'],inplace=True)
bath_protein['ID'] = bath_protein['query name']

<ipython-input-12-94347e72fdd2>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bath_protein['start'].fillna(bath_protein['ali from flip'],inplace=True)
<ipython-input-12-94347e72fdd2>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

### Remove IS elements from both Prokka and BATH data

In [13]:
bath_protein = bath_protein[~bath_protein['query name'].str.startswith("IS")]

In [14]:
prokka_proteins = prokka_proteins[~prokka_proteins['inference'].str.contains("ISfinder")]

In [15]:
print(bath_protein.shape,prokka_proteins.shape)

(29015, 27) (75416, 20)


### Save new bath and prokka protein files

In [ ]:
bath_protein.to_excel(bath_dedup_annotations_save,index=False)
prokka_proteins.to_excel(prokka_annotations_save,index=False)

## Find the gaps filled by BATH vs Prokka per contig

In [ ]:
# Separate BATH data into positive and negative strands
pos_strand = bath_protein[bath_protein['strand'] == "+"].copy()
neg_strand = bath_protein[bath_protein['strand'] == "-"].copy()

# Handle columns for positive strand
pos_strand.loc[:, 'start'] = pos_strand['ali from'].astype(int)
pos_strand.loc[:, 'end'] = pos_strand['ali to'].astype(int)

# Handle columns for negative strand
neg_strand.loc[:, 'start'] = neg_strand['ali from flip'].astype(int)
neg_strand.loc[:, 'end'] = neg_strand['ali to flip'].astype(int)

# Filter Prokka proteins into positive and negative strands
prokka_proteins_plus = prokka_proteins[prokka_proteins['strand'] == '+'].copy()
prokka_proteins_minus = prokka_proteins[prokka_proteins['strand'] == '-'].copy()

# Sort and save positive strand Prokka proteins
prokka_proteins_plus.sort_values('start', inplace=True)
prokka_proteins_plus.to_csv('prokka_proteins_plus.bed', sep='\t', index=False, header=False, columns=['seq_id', 'start', 'end', 'strand', 'ID', 'inference'])

# Sort and save negative strand Prokka proteins
prokka_proteins_minus.sort_values('start', inplace=True)
prokka_proteins_minus.to_csv('prokka_proteins_minus.bed', sep='\t', index=False, header=False, columns=['seq_id', 'start', 'end', 'strand', 'ID', 'inference'])

# Sort and save BATH deduplicated positive strand
bath_deduplicated_plus = pos_strand[['target name', 'start', 'end', 'strand', 'ID', 'shifts','E-value']].copy()
bath_deduplicated_plus.sort_values('start', inplace=True)
bath_deduplicated_plus.to_csv('bath_deduplicated_plus.bed', sep='\t', index=False, header=False)

# Sort and save BATH deduplicated negative strand
bath_deduplicated_minus = neg_strand[['target name', 'start', 'end', 'strand', 'ID', 'shifts','E-value']].copy()
bath_deduplicated_minus.sort_values('start', inplace=True)
bath_deduplicated_minus.to_csv('bath_deduplicated_minus.bed', sep='\t', index=False, header=False)


In [ ]:
print(bath_deduplicated_plus.shape,bath_deduplicated_minus.shape)
print(prokka_proteins_plus.shape,prokka_proteins_minus.shape)

### Functions



1.   generate_bed_files:
This function creates BED files by iterating through unique contig identifiers in the DataFrame, filtering and sorting data by start positions, and writing the data to BED files.

2.   run_bedmap_operations:
This function uses bedmap and bedops tools to find overlapping and unique regions between BATH and Prokka BED files. It generates three output files: one for overlaps, one for regions unique to BATH, and one for regions unique to Prokka, saving them in the specified output directory

3.   find_and_process_bed_pairs:
This function finds and pairs BED files from BATH and Prokka directories based on sequence identifiers. It then calls run_bedmap_operations to process each pair, identifying overlaps and unique regions, and saves the results in the output directory.

4.   combine_files_by_prefix:
This function organizes and combines BED files in a source directory by their prefixes. It groups files with the same prefix, concatenates their contents, and writes the combined data into new BED files in the output directory. Each new file is named with the prefix followed by "combined.bed".

In [ ]:
#Functions

def generate_bed_files(df, directory, contig_column, file_prefix='Prokka_annotation'):
    # Create directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)

    total_lines_processed = 0  # Initialize a counter for total lines processed

    # Loop over unique sequence identifiers (contig_column)
    for seq_id in df[contig_column].unique():
        df_seq = df[df[contig_column] == seq_id]
        df_seq = df_seq.sort_values(by=['start'])
        filename = f'{directory}/{file_prefix}_{seq_id}.bed'

        # Initialize a counter for this seq_id
        lines_processed = 0

        # Write BED file for this seq_id
        with open(filename, 'w') as file:
            for _, row in df_seq.iterrows():
                if file_prefix == "Prokka_annotation":
                    bed_line = f"{row[contig_column]}\t{row['start']}\t{row['end']}\t{row['ID']}\t{row['strand']}\t{row['inference']}\n"
                else:
                    bed_line = f"{row[contig_column]}\t{row['start']}\t{row['end']}\t{row['ID']}\t{row['strand']}\t{row['shifts']}\t{row['E-value']}\n"
                file.write(bed_line)
                lines_processed += 1  # Increment the counter for this seq_id

        # Update the total lines processed across all files
        total_lines_processed += lines_processed

        # Print the number of lines processed for this specific seq_id
        print(f"Processed {lines_processed} lines for {seq_id}")

    # Print the total number of lines processed
    print(f"Total lines processed: {total_lines_processed}")
    print(f"BED files have been successfully generated in {directory}")

def run_bedmap_operations(bath_bed_path, prokka_bed_path, output_base_dir):
    seq_id = bath_bed_path.stem.split('annotation_')[-1]
    print("Processing this:",seq_id)
    overlap_output = output_base_dir / f'overlap_prokka_bath_{seq_id}.bed'
    unique_to_bath_output = output_base_dir / f'unique_to_bath_{seq_id}.bed'
    unique_to_prokka_output = output_base_dir / f'unique_to_prokka_{seq_id}.bed'

    output_base_dir.mkdir(parents=True, exist_ok=True)

    # Command for bedmap to find overlaps
    command = [
        "bedmap", "--echo", "--echo-map", "--delim", ";",
        "--fraction-ref", "0.15",
        str(bath_bed_path), str(prokka_bed_path)
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True, check=True)

    # Filter the output and write to overlap_output
    with open(overlap_output, 'w') as f_output:
        for line in result.stdout.splitlines():
            parts = line.split(';')
            # Check if there is more than one part and the second part is not empty
            if len(parts) > 1 and parts[1].strip():
                f_output.write(line + '\n')

    # Run bedmap for unique to BATH
    subprocess.run(["bedops", "--not-element-of", "1", str(bath_bed_path), str(prokka_bed_path)], stdout=open(unique_to_bath_output, 'w'), check=True)

    # Run bedmap for unique to Prokka
    subprocess.run(["bedops", "--not-element-of", "1", str(prokka_bed_path), str(bath_bed_path)], stdout=open(unique_to_prokka_output, 'w'), check=True)

def find_and_process_bed_pairs(bath_dir, prokka_dir, output_dir):
    bath_dir_path = Path(bath_dir)
    prokka_dir_path = Path(prokka_dir)
    output_base_dir = Path(output_dir)

    # Extract the sequence identifier by removing the prefix and taking the relevant part of the filename.
    bath_bed_files = {f.stem.replace('BATH_annotation_', ''): f for f in bath_dir_path.glob('BATH_annotation_*.bed')}
    prokka_bed_files = {f.stem.replace('Prokka_annotation_', ''): f for f in prokka_dir_path.glob('Prokka_annotation_*.bed')}

    for seq_id, bath_bed_path in bath_bed_files.items():
        print("Processing sequence ID:", seq_id)
        prokka_bed_path = prokka_bed_files.get(seq_id)
        if prokka_bed_path:
            print("Found matching files: ", prokka_bed_path, " and ", bath_bed_path)
            run_bedmap_operations(bath_bed_path, prokka_bed_path, output_base_dir)
        else:
            print(f"No matching Prokka file found for BATH seq_id: {seq_id}")

def combine_files_by_prefix(source_directory, output_directory, prefixes):
    source_directory = Path(source_directory)
    output_directory = Path(output_directory)
    output_directory.mkdir(parents=True, exist_ok=True)
    file_groups = {prefix: [] for prefix in prefixes}
    for file_path in source_directory.glob('*.bed'):
        for prefix in prefixes:
            if file_path.name.startswith(prefix):
                file_groups[prefix].append(file_path)
    for prefix, files in file_groups.items():
        combined_file_path = output_directory / f"{prefix}combined.bed"
        with combined_file_path.open('w') as combined_file:
            for file_path in files:
                with file_path.open() as input_file:
                    combined_file.write(input_file.read())
                combined_file.write('\n')

In [ ]:
import os

BASE_DIR = '/content/BED'
PROKKA_PLUS_DIR = f'{BASE_DIR}/Prokka_Annotation_PLUS_BED'
PROKKA_MINUS_DIR = f'{BASE_DIR}/Prokka_Annotation_MINUS_BED'
BATH_PLUS_DIR = f'{BASE_DIR}/BATH_Annotation_PLUS_BED'
BATH_MINUS_DIR = f'{BASE_DIR}/BATH_Annotation_MINUS_BED'
OVERLAPS_DIR = f'{BASE_DIR}/Overlaps and Differences'
COMBINED_FILES_DIR = f'{BASE_DIR}/Combined Files'


directories = [
    PROKKA_PLUS_DIR, PROKKA_MINUS_DIR,
    BATH_PLUS_DIR, BATH_MINUS_DIR,
    OVERLAPS_DIR, COMBINED_FILES_DIR
]
for directory in directories:
    os.makedirs(directory, exist_ok=True)


generate_bed_files(prokka_proteins_plus, PROKKA_PLUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
generate_bed_files(prokka_proteins_minus, PROKKA_MINUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
generate_bed_files(bath_deduplicated_plus, BATH_PLUS_DIR, 'target name', file_prefix='BATH_annotation')
generate_bed_files(bath_deduplicated_minus, BATH_MINUS_DIR, 'target name', file_prefix='BATH_annotation')


find_and_process_bed_pairs(BATH_PLUS_DIR, PROKKA_PLUS_DIR, OVERLAPS_DIR)

prefixes = ['unique_to_prokka_', 'unique_to_bath_', 'overlap_prokka_bath_']
combine_files_by_prefix(OVERLAPS_DIR, COMBINED_FILES_DIR, prefixes)

### Analysis

#### Venn diagram of region based overlaps

In [ ]:
def transform_file_to_dataframe(file_path):
    print(f"Processing file: {file_path}")  # Display the file being processed

    transformed_rows = []
    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Check if the file is empty
        if not lines:
            raise ValueError(f"The file '{file_path}' is empty.")

        for line in lines:
            fields = line.strip().split(';')

            # Ensure that the fields follow the expected format (adjust as needed for your case)
            if len(fields) == 3:
                new_row1 = [fields[0], fields[1]]  # BATH and Prokka match
                new_row2 = [fields[0], fields[2]]  # Add another matching pair if needed
                transformed_rows.append(new_row1)
                transformed_rows.append(new_row2)
            else:
                # Handle lines with fewer/more fields if necessary (adjust this based on your file format)
                transformed_rows.append(fields)

    # Filter out invalid rows (those that don't match expected length) and create DataFrame
    valid_rows = [row for row in transformed_rows if len(row) == 2]
    df = pd.DataFrame(valid_rows, columns=['BATH', 'Prokka']).replace("\n", np.nan).dropna()

    return df


In [ ]:
file_path = '/content/BED/Combined Files/overlap_prokka_bath_combined.bed'
overlaps = transform_file_to_dataframe(file_path)

In [ ]:
def determine_label(row):
    if row['Prokka'].endswith('ab initio prediction:Prodigal:002006'):
        return "BATH and Prokka Unannotated"
    else:
        return "BATH and Prokka Annotated"

overlaps['label'] = overlaps.apply(determine_label, axis=1)

In [ ]:
file_path = '/content/BED/Combined Files/unique_to_bath_combined.bed'
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
unique_to_bath = pd.DataFrame(lines, columns=['BATH']).replace("\n",np.nan).dropna()
unique_to_bath['Prokka'] = np.nan
unique_to_bath['label'] = 'BATH'

In [ ]:
file_path3 = '/content/BED/Combined Files/unique_to_prokka_combined.bed'
with open(file_path3, 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
unique_to_prokka = pd.DataFrame(lines, columns=['Prokka']).replace("\n",np.nan).dropna()
unique_to_prokka['BATH'] = np.nan

def determine_label2(row):
    if row['Prokka'].endswith('ab initio prediction:Prodigal:002006'):
        return "Prokka Unannotated"
    else:
        return "Prokka Annotated"

unique_to_prokka['label'] = unique_to_prokka.apply(determine_label2, axis=1)

In [ ]:
venn_data = pd.concat([unique_to_prokka,unique_to_bath,overlaps])

In [ ]:
venn_data['label'].value_counts()

## Remove alignments identified by Prokka and not BATH because they are closer to the boundary

In [ ]:
#Take data from prokka annotated and now go back to prededuplicated data to find alignments to this and then capture the lowest E-value, if this E-value is higher than the threshold then throw away this prokka data
only_prokka_venn = venn_data[venn_data['label'] == "Prokka Annotated"].copy()

In [ ]:
header = [
    "target name", "accession", "query name", "accession", "hmm len",
    "hmm from", "hmm to", "seq len", "ali from", "ali to",
    "env from", "env to", "E-value", "score", "bias",
    "shifts", "stops", "pipe", "description of target"
]

dataframes = []
for i in list_of_bathsearch_outputs:
    df = pd.read_csv(i, skipfooter=9, skiprows=2, header=None, sep="\s+", engine='python')
    dataframes.append(df)
    #print(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.columns = header
combined_df.to_csv(bathsearch_output_file, index=False)

combined_df

In [ ]:
# Split the 'Prokka' column by '\t' and expand into new columns
expanded_cols = only_prokka_venn['Prokka'].str.split('\t', expand=True)
expanded_cols.columns = ['seq_id', 'start', 'end', 'ID', 'strand', 'inference']
only_prokka_venn = pd.concat([expanded_cols, only_prokka_venn], axis=1)

In [ ]:
print(only_prokka_venn.columns)
print(only_prokka_venn.columns.duplicated())

In [ ]:
only_prokka_venn

In [ ]:
only_prokka_venn_pos = only_prokka_venn[only_prokka_venn['strand'] == "+"]
only_prokka_venn_neg = only_prokka_venn[only_prokka_venn['strand'] == "-"]

combined_df['strand'] = np.where(combined_df['ali from'] - combined_df['ali to'] < 0, "-", "+")
combined_df['start'] = combined_df['ali from']
combined_df['end'] = combined_df['ali to']
combined_df['ID'] = combined_df['query name']
combined_df_pos = combined_df[combined_df['strand']=="+"]
combined_df_neg = combined_df[combined_df['strand']=="-"]


In [ ]:
# Define directories for separate analysis
SEPARATE_ANALYSIS_DIR = f'{BASE_DIR}/Separate Analysis'
SEPARATE_PROKKA_PLUS_DIR = f'{SEPARATE_ANALYSIS_DIR}/Prokka_Annotation_PLUS_BED'
SEPARATE_PROKKA_MINUS_DIR = f'{SEPARATE_ANALYSIS_DIR}/Prokka_Annotation_MINUS_BED'
SEPARATE_BATH_PLUS_DIR = f'{SEPARATE_ANALYSIS_DIR}/BATH_Annotation_PLUS_BED'
SEPARATE_BATH_MINUS_DIR = f'{SEPARATE_ANALYSIS_DIR}/BATH_Annotation_MINUS_BED'
SEPARATE_OVERLAPS_DIR = f'{SEPARATE_ANALYSIS_DIR}/Overlaps and Differences'
SEPARATE_Combined_DIR = f'{SEPARATE_ANALYSIS_DIR}/Combined Files'
SEPARATE_OVERLAPS_DIR = f'{SEPARATE_ANALYSIS_DIR}/Overlaps and Differences'
SEPARATE_Combined_DIR_OPP = f'{SEPARATE_ANALYSIS_DIR}/Combined Files Opposite Strands'
SEPARATE_OVERLAPS_DIR_OPP = f'{SEPARATE_ANALYSIS_DIR}/Overlaps and Differences Opposite Strands'

In [ ]:
separate_directories = [
    SEPARATE_ANALYSIS_DIR, SEPARATE_PROKKA_PLUS_DIR, SEPARATE_PROKKA_MINUS_DIR,
    SEPARATE_BATH_PLUS_DIR, SEPARATE_BATH_MINUS_DIR, SEPARATE_OVERLAPS_DIR, SEPARATE_Combined_DIR,
    SEPARATE_Combined_DIR_OPP, SEPARATE_OVERLAPS_DIR_OPP
]

for directory in separate_directories:
    os.makedirs(directory, exist_ok=True)

# Generate BED files for Prokka and BATH annotations for both positive and negative strands in the new directories
generate_bed_files(only_prokka_venn_pos, SEPARATE_PROKKA_PLUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
generate_bed_files(only_prokka_venn_neg, SEPARATE_PROKKA_MINUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
generate_bed_files(combined_df_pos, SEPARATE_BATH_PLUS_DIR, 'target name', file_prefix='BATH_annotation')
generate_bed_files(combined_df_neg, SEPARATE_BATH_MINUS_DIR, 'target name', file_prefix='BATH_annotation')

# Process BED pairs to find overlaps and unique regions between BATH and Prokka in the new directories
find_and_process_bed_pairs(SEPARATE_BATH_PLUS_DIR, SEPARATE_PROKKA_PLUS_DIR, SEPARATE_OVERLAPS_DIR)
find_and_process_bed_pairs(SEPARATE_BATH_MINUS_DIR, SEPARATE_PROKKA_MINUS_DIR, SEPARATE_OVERLAPS_DIR)

#Process opposite strand files
find_and_process_bed_pairs(SEPARATE_BATH_MINUS_DIR, SEPARATE_PROKKA_PLUS_DIR, SEPARATE_OVERLAPS_DIR_OPP)
find_and_process_bed_pairs(SEPARATE_BATH_PLUS_DIR, SEPARATE_PROKKA_MINUS_DIR, SEPARATE_OVERLAPS_DIR_OPP)

# Combine resulting BED files by specified prefixes in the new directories
prefixes = ['unique_to_prokka_', 'unique_to_bath_', 'overlap_prokka_bath_']
combine_files_by_prefix(SEPARATE_OVERLAPS_DIR, SEPARATE_Combined_DIR, prefixes)
combine_files_by_prefix(SEPARATE_OVERLAPS_DIR_OPP, SEPARATE_Combined_DIR_OPP, prefixes)

# Annotations with hits on -ve strands

In [ ]:
file_path_prokka_opposite_strand = '/content/BED/Separate Analysis/Combined Files Opposite Strands/overlap_prokka_bath_combined.bed'
overlaps_prokka_opposite_strand = transform_file_to_dataframe(file_path_prokka_opposite_strand)
overlaps_prokka_opposite_strand

In [ ]:
overlaps_prokka_opposite_strand.groupby(['Prokka']).agg(";".join)

In [ ]:
columns_to_drop = overlaps_prokka_opposite_strand.groupby(['Prokka']).agg(";".join).index.unique()

In [ ]:
venn_data_filtered = venn_data[~venn_data['Prokka'].isin(columns_to_drop)]
print(venn_data_filtered)

# Annotations with hits having E -values closer to threshold



In [ ]:
file_path_prokka_only_same_strand = '/content/BED/Separate Analysis/Combined Files/overlap_prokka_bath_combined.bed'
overlaps_prokka_only_same_strand = transform_file_to_dataframe(file_path_prokka_only_same_strand)
overlaps_prokka_only_same_strand

In [ ]:
def filter_venn_data(overlaps_prokka_only_same_strand):
    filtered_prokka_values = []
    for index, row in overlaps_prokka_only_same_strand.iterrows():
        bath_value = float(row['BATH'].split("\t")[-1])
        if bath_value > 10**-6:
            filtered_prokka_values.append(row['Prokka'])

    filtered_prokka_values = list(set(filtered_prokka_values))

    return filtered_prokka_values


In [ ]:
filtered_prokka_values = filter_venn_data(overlaps_prokka_only_same_strand)

In [ ]:
venn_data_filtered2 = venn_data_filtered[~venn_data_filtered['Prokka'].isin(filtered_prokka_values)]

# Measuring Overlaps

In [ ]:
prokka_unannotated = venn_data_filtered2[venn_data_filtered2['label'] == "Prokka Unannotated"].shape[0]
only_bath = venn_data_filtered2[venn_data_filtered2['label'] == "BATH"].shape[0]
intersection_3 = venn_data_filtered2[venn_data_filtered2['label'] == "BATH and Prokka Annotated"].shape[0]
bath_prokka_unannotated = venn_data_filtered2[venn_data_filtered2['label'] == "BATH and Prokka Unannotated"].shape[0] + venn_data_filtered2[venn_data_filtered2['label'] == "BATH and Prokka Annotated"].shape[0]
prokka_unannot_annot = venn_data_filtered2[venn_data_filtered2['label'] == "Prokka Annotated"].shape[0]


In [ ]:
from matplotlib_venn import venn3
import matplotlib.pyplot as plt

plt.style.use('seaborn')
venn_counts = {
    '100': only_bath,  # Only BATH
    '010': 0,  # Only Prokka Annotated (Nothing since everything is prodigal and prokka annotated)
    '001': prokka_unannotated,  # Only Prokka Unannotated
    '110': 0,  # Intersection BATH and Prokka Annotated (Nothing since everything is prodigal and prokka annotated)
    '101': venn_data_filtered2[venn_data_filtered2['label'] == "BATH and Prokka Unannotated"].shape[0],  # Intersection BATH and Prokka Unannotated
    '011': prokka_unannot_annot,  # Intersection of Prokka Annotated and Prokka Unannotated (not given)
    '111': intersection_3   # Intersection of all three (not applicable)
}


set_colors = ("#de3028", "#1B263B", "#FFD700")
venn3(subsets=venn_counts, set_labels=('BATH Annotations', 'Prokka Annotations', 'ORFs (Prodigal)'), set_colors=set_colors)


plt.title("Overlaping annotations BATH and Prokka based DNA position")
plt.show()


In [ ]:
venn_data_filtered2.to_excel(bath_prokka_alignment,index=False)